# Damped bead on a rotating Hoop

From Sec. 3.5 in Strogatz.  Full equation:
$$
m R \ddot{\phi} = -b \dot{\phi} - m g \sin \phi + m R^2 \omega \sin \phi \cos \phi
$$

If we introduce nondimensionalizations
$$
\gamma = \frac{R \omega^2}{g} \hspace{2cm} \epsilon = \frac{m^2 g R}{b^2}  \hspace{2cm} \tau = \frac{t}{T} = \frac{b}{mg},
$$
the equation becomes
$$
\epsilon \frac{d^2 \phi}{d \tau^2} = - \frac{d \phi}{d \tau} - \sin \phi + \gamma \sin \phi \cos \phi.
$$
For $\epsilon \ll 1$ and $\gamma = \mathcal{O}(1)$, the system is overdamped and approximately first-order.  The system undergoes a pitchfork bifurcation at $\gamma = 1$.

In [48]:
# Matplotlib
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import rcParams
mpl.rc('text', usetex=True)
mpl.rc('font', family='serif')
mpl.rc('xtick', labelsize=14)
mpl.rc('ytick', labelsize=14)
mpl.rc('axes', labelsize=20)
mpl.rc('axes', titlesize=20)
mpl.rc('figure', figsize=(6, 4))
%config InlineBackend.figure_format = 'retina'

from numpy.linalg import matrix_rank
import sys
sys.path.append('../src')
sys.path.append('../solvers')
from rotating_hoop import RotatingHoop
from learning import KRidgeReg, NeuralNet, BuckyNet, KRidgeReg_struct
from nullspace_search import get_nondim_numbers, fit_allnondim
from helper_functions import prettify_results


## Get Rotating hoop data

In [57]:
nsamples = int(6e3)
output_type = 'svd' # options: 'dynamic', 'static', 'svd' - 
num_nondim = 3 # !! Recomputed below - num_nondim=3 also works for 'svd' in some cases !!
num_modes = 5
tsteps = 100 # For 'dynamic', use smaller number of tsteps - dynamic still takes too long
tend = 2
phi_init = [1, 0]

## Get solution
R = RotatingHoop(nsamples=nsamples, output_type=output_type, modes=num_modes, time_steps=tsteps, tend=tend, phi0=phi_init)
inputs, outputs = R.get_data()
dim_matrix = R.get_dim_matrix()


In [58]:
inputs.shape

(6000, 5)

In [63]:
## TODO: Set up hyperparameter search.

verbose = 1
num_layers = 3
num_neurons = 80
activation = 'elu'
initializer = 'he_normal'
nepoch = 50000
patience = 100
test_size = 0.2
nullspace_loss = .8 # Set to weight \in [0, 1] if want to turn on
l1_reg = 0.000
l2_reg = 0.000
adamlr = 0.002

B = BuckyNet(inputs, outputs, dim_matrix, num_nondim=num_nondim, num_layers=num_layers, num_neurons=num_neurons, activation=activation, verbose=verbose, initializer=initializer, nepoch=nepoch, 
             patience=patience, test_size=test_size, nullspace_loss=nullspace_loss, l1_reg=l1_reg, l2_reg=l2_reg, adamlr=adamlr)


In [64]:
x = B.single_run()

Epoch 1/50000
120/120 [==============================] - 0s 1ms/step - loss: 0.4186 - root_mean_squared_error: 0.4137 - val_loss: 0.1786 - val_root_mean_squared_error: 0.0541
Epoch 2/50000
120/120 [==============================] - 0s 677us/step - loss: 0.1364 - root_mean_squared_error: 0.1109 - val_loss: 0.1006 - val_root_mean_squared_error: 0.1319
Epoch 3/50000
120/120 [==============================] - 0s 671us/step - loss: 0.0638 - root_mean_squared_error: 0.0774 - val_loss: 0.0388 - val_root_mean_squared_error: 0.0395
Epoch 4/50000
120/120 [==============================] - 0s 674us/step - loss: 0.0285 - root_mean_squared_error: 0.0541 - val_loss: 0.0221 - val_root_mean_squared_error: 0.0726
Epoch 5/50000
120/120 [==============================] - 0s 673us/step - loss: 0.0148 - root_mean_squared_error: 0.0505 - val_loss: 0.0120 - val_root_mean_squared_error: 0.0573
Epoch 6/50000
120/120 [==============================] - 0s 676us/step - loss: 0.0094 - root_mean_squared_error: 0.05

In [65]:
# Need to find a way to do these automatically
# Say, we can divide by the largest number in the vector as a start (or simply avoid the smaller ones, the ones close to zero)
# Or as an over-kill, we can find the combination that has the least non-zero fractions.

idxs = [1, 1, -1]

x1_norm = x[:, 0]/x[idxs[0], 0]
x2_norm = x[:, 1]/x[idxs[1], 1]
x3_norm = x[:, 2]/x[idxs[2], 2]
Pi, names = R.get_dim_matrix(include_names=True)

print(x)
print('-------------')
print(x1_norm)
print(x2_norm)
print(x3_norm)
print('-------------')
print(x@Pi)


[[ 1.1825006e+00  9.1652926e-03  6.9539662e-04]
 [ 5.8654070e-01  5.6169885e-01  1.7913679e-03]
 [-1.1824903e+00 -9.1935573e-03 -6.9167762e-04]
 [ 5.9601086e-01 -5.5255842e-01 -1.0809802e-03]
 [-9.4192373e-03  1.1142274e+00  2.8202247e-03]]
-------------
[ 2.016059    1.         -2.0160415   1.0161458  -0.01605897]
[ 0.01631709  1.         -0.01636741 -0.98372716  1.983674  ]
[ 0.2465749   0.6351862  -0.2452562  -0.38329574  1.        ]
-------------
[[ 9.16529261e-03  1.18250060e+00  1.19097050e+00  1.18110981e+00
  -6.95396622e-04]
 [ 5.61698854e-01  5.86540699e-01  1.14644819e+00  5.82957963e-01
  -1.79136789e-03]
 [-9.19355731e-03 -1.18249035e+00 -1.19099223e+00 -1.18110699e+00
   6.91677618e-04]
 [-5.52558422e-01  5.96010864e-01  4.45334219e-02  5.98172824e-01
   1.08098018e-03]
 [ 1.11422741e+00 -9.41923726e-03  1.10198795e+00 -1.50596867e-02
  -2.82022473e-03]]


In [66]:
from helper_functions import prettify_results

prettify_results(x1_norm, names, tol=0.1, max_degree=6)
prettify_results(x2_norm, names, tol=0.1, max_degree=6)
prettify_results(x3_norm, names, tol=0.1, max_degree=6)

## Known result is not guaranteed, but loss seems to be a good indicator

[ 2.016059    1.         -2.0160415   1.0161458  -0.01605897]
m : 2
R : 1
b : -2
g : 1
w : 0


<IPython.core.display.Math object>

[ 0.01631709  1.         -0.01636741 -0.98372716  1.983674  ]
m : 0
R : 1
b : 0
g : -1
w : 2


<IPython.core.display.Math object>

[ 0.2465749   0.6351862  -0.2452562  -0.38329574  1.        ]
m : 0.3333333333333333
R : 0.6666666666666666
b : -0.3333333333333333
g : -0.3333333333333333
w : 1


<IPython.core.display.Math object>